### Clonamos o diretório darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

### entrando no diretório do darknet

In [ ]:
cd darknet

### caso tenha uma GPU com suporte a CUDA abra o arquivo Makefile e mude as propriedades GPU=0 para 1, CUDNN=0 para 1, CUDNN_HALF=1 e compile o modelo usando o comando abaixo

In [ ]:
!make

1. crie um arquivo myprog.cfg com o mesmo conteudo do arquivo yolov4-custom.cfg

2. mude o valor de max_batches para <numero de classes> * 2000

3. mude o valor de step para "80% do valor de max_batches" , "90% do valor de max_batches"

4. mude width e height para um valor multiplo de 32. Ex: 416

5. nos 3 blocos [convolutional] mude o valor de filters de 255 para (classes + 5)x3 e dos 3 blocos [yolo] o valor de classes para o seu numero de classes

6. crie um arquivo myprog.cfg em build\darknet\x64\data\ com o nome das classes, cada um por linha.

7. crie um arquivo myprog.data em build\darknet\x64\data\ configurando de acorde com sua necessidade no seguinte formato:<br>
   classes = (numero de classes)<br>
   train  = caminho/para/arquivo/train.txt<br>
   valid  = caminho/para/arquivo/test.txt<br>
   names = caminho/para/myprog.names<br>
   backup = backup/

8. coloque suas imagens em build\darknet\x64\data\myprog\ use uma ferramento como o labelImg ou o label-studio para criar o seu dataset rotulado de forma simples para o yolo.
Com uma dessas ferramentas vc tera as imagens acompanhadas de arquivos txt contendo as cordenadas da bounding box e a classe
no formato "object-class" "x_center" "y_center" "width" "height"

9. crie um arqui train.txt e test.txt em build\darknet\x64\data\ contendo os caminho completo para as imagens correspondentes

10. baixe os pesos do [yolo4.conv.137](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137) e coloque em build\darknet\x64

### estando tudo configurado use o comando para iniciar o treinamento
obs: no meu computador (i5 2500k + 16RAM sem GPU com CUDA)o treinamento foi previsto para uma estimativa de quase 1 semanda então tenha em mente que dependendo da sua maquina pode demorar bastante.

In [ ]:
!./darknet detector train build/darknet/x64/data/myprog.data cfg/myprog.cfg build/darknet/x64/yolov4.conv.137

### Testando

In [ ]:
!./darknet detector map build/darknet/x64/data/myprog.data cfg/myprog.cfg build/darknet/x64/yolov4.conv.137

### para usar o modelo treinado em uma imagem especifica use o seguinte comando:

In [ ]:
!python darknet_images.py --weights ./backup/myprog_last.weights \
                         --data_file build/darknet/x64/data/myprog.data \
                         --config_file cfg/myprog.cfg \
                         --input /caminho/para/img
